<a href="https://colab.research.google.com/github/zooodung/TA-Prediction/blob/main/final_onlycode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 교통사고 위험도 예측

## 라이브러리 및 파일 호출

In [ ]:
!pip install pytimekr

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pytimekr import pytimekr
from sklearn.preprocessing import LabelEncoder
import os

In [ ]:
path='/content/drive/MyDrive/camp/project1/open/'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
daegu_data=pd.read_csv(path+'train.csv')
wide_data=pd.read_csv(path+'external_open/countrywide_accident.csv')
test_data=pd.read_csv(path+'test.csv')
daegu_data.head()

## 기초 통계 분석

In [ ]:
#ta_data_daegu.info()

In [ ]:
#x_test.info()

In [ ]:
#ta_data_daegu.describe()

In [ ]:
#ta_data_daegu.describe()

In [ ]:
#columns = ['요일', '기상상태', '도로형태', '노면상태', '사고유형']

#for column in columns:
#    print(f"Column: {column}")
#    print(ta_data_daegu[column].value_counts())
#    print("-" * 30)

## 전처리

### 대구 데이터 전처리

#### 불필요한 컬럼 제거

In [ ]:
columns_to_drop = ['ID', '사고유형 - 세부분류', '법규위반', '가해운전자 차종', '가해운전자 성별', '가해운전자 연령',
                   '가해운전자 상해정도', '피해운전자 차종', '피해운전자 성별', '피해운전자 연령', '피해운전자 상해정도']


daegu=daegu_data.drop(columns=columns_to_drop)

#### 사고일시 컬럼 연도, 월, 일, 시간 단위로 분할

In [ ]:
#daegu[['년도','월','일+시간']] = daegu['사고일시'].str.split('-',expand=True)
#daegu[['일','시간']]= daegu['일+시간'].str.split(' ',expand=True)

#daegu.drop(['일+시간','사고일시'],inplace=True, axis=1)
#daegu.drop(['일'],inplace=True, axis=1)

#### 시군구 단위로 분할

In [ ]:
daegu[['시','구','동']] = daegu['시군구'].str.split(' ',expand=True)
daegu.drop(['시군구'],inplace=True, axis=1)

#### 도로 형태 분할

In [ ]:
daegu[['도로형태1', '도로형태2']] = daegu['도로형태'].str.split(' - ', expand=True)
daegu.drop(['도로형태'],inplace=True, axis=1)

In [ ]:
daegu.head()

#### 요일 컬럼 평일과 주말로 분류

In [ ]:
#ta_data_daegu['요일 분류'] = ta_data_daegu['요일'].apply(lambda x: '평일' if x in ['월요일', '화요일', '수요일', '목요일', '금요일'] else '주말')

In [ ]:
#ta_data_daegu

### 전국 데이터 전처리

In [ ]:
wide=wide_data.copy()
len(wide)

#### 철길 건널목 제거

In [ ]:
wide = wide[~wide['도로형태'].str.split().str[-1].str.contains('철길건널목')]

#### 불필요한 컬럼 제거

In [ ]:
wide=wide.drop(columns=columns_to_drop)

#### 광역시, 특별시만 분류

In [ ]:
def contains_specific_location(address):
    cities = ['서울특별시', '인천광역시', '부산광역시', '울산광역시', '광주광역시', '대전광역시']
    for city in cities:
        if city in address:
            return True
    return False
wide=wide[wide['시군구'].apply(contains_specific_location)]

wide[['시','구','동']] = wide['시군구'].str.split(' ',expand=True)

In [ ]:
wide['시군구'].apply(lambda x:x.split()[0]).value_counts()

In [ ]:
wide.drop(['시군구'],inplace=True, axis=1)

#### 연도, 월, 일, 시간 분류

In [ ]:
#wide[['년도','월','일+시간']] = wide['사고일시'].str.split('-',expand=True)
#wide[['일','시간']]= wide['일+시간'].str.split(' ',expand=True)

#wide.drop(['일+시간','사고일시'],inplace=True, axis=1)
#wide.drop(['일'],inplace=True, axis=1)

#### 도로 형태 분류

In [ ]:
wide[['도로형태1', '도로형태2']] = wide['도로형태'].str.split(' - ', expand=True)

wide.drop(['도로형태'],inplace=True, axis=1)

In [ ]:
wide.head()

### 데이터 병합 후 전처리 진행

In [ ]:
print(len(daegu),len(wide), len(daegu)+len(wide))

In [ ]:
concatenated_df = pd.concat([daegu, wide])
concatenated_df

#### 휴일 여부

In [ ]:
for i in range(2019, 2024):
    kr_holidays = pytimekr.holidays(i)
    print(kr_holidays)

In [ ]:
kr_holidays = ['2019-01-01', '2019-02-04', '2019-02-05', '2019-02-06', '2019-03-01', '2019-05-05', '2019-05-12', '2019-06-06', '2019-08-15', '2019-09-12', '2019-09-13', '2019-09-14', '2019-10-03', '2019-10-09', '2019-12-25',
                '2020-01-01' ,'2020-01-24' ,'2020-01-25', '2020-01-26', '2020-03-01', '2020-04-30', '2020-05-05', '2020-06-06', '2020-08-15', '2020-08-17', '2020-09-30', '2020-10-01', '2020-10-02', '2020-10-03', '2020-10-09', '2020-12-25',
                '2021-01-01' ,'2021-02-11' ,'2021-02-12', '2021-02-13', '2021-03-01', '2021-05-05', '2021-05-19', '2021-06-06', '2021-08-15', '2021-09-20', '2021-09-21', '2021-09-22', '2021-10-03', '2021-10-09', '2021-12-25',
                '2022-01-01' ,'2022-01-31' ,'2022-02-01', '2022-02-02', '2022-03-01', '2022-05-05', '2022-05-08', '2022-06-06', '2022-08-15', '2022-09-09', '2022-09-10', '2022-09-11', '2022-09-12', '2022-10-03', '2022-10-09', '2020-10-10', '2022-12-25',
                '2023-01-01' ,'2023-01-21' ,'2023-01-22', '2023-01-23', '2023-01-24', '2023-03-01', '2023-05-05', '2023-05-26', '2023-06-06', '2023-08-15', '2023-09-28', '2023-09-29', '2023-09-30', '2023-10-03', '2023-10-09', '2023-12-25']


In [ ]:
concatenated_df['사고일시'] = pd.to_datetime(concatenated_df['사고일시'])

# datetime에서 year, month, day, hour 추출하여 새로운 열 생성
concatenated_df['년도'] = concatenated_df['사고일시'].dt.year
concatenated_df['월'] = concatenated_df['사고일시'].dt.month
concatenated_df['day'] = concatenated_df['사고일시'].dt.day
concatenated_df['시간'] = concatenated_df['사고일시'].dt.hour
concatenated_df['day_of_week'] = concatenated_df['사고일시'].dt.dayofweek

In [ ]:
concatenated_df['holiday'] = np.where((concatenated_df.day_of_week >= 5) | (concatenated_df.사고일시.dt.strftime('%Y-%m-%d').isin(kr_holidays)), '휴일', '평일')

In [ ]:
concatenated_df.drop(columns=['사고일시'], inplace=True)

In [ ]:
concatenated_df['holiday'].value_counts()

In [ ]:
concatenated_df.drop(['day', 'day_of_week'], inplace=True, axis=1)

#### 주말 여부

In [ ]:
#concatenated_df['주말여부']=concatenated_df['요일'].apply(lambda x:'주말' if x in ['토요일','일요일'] else '평일')

#concatenated_df['주말여부'].value_counts()

#### 시간 분류

In [ ]:
def seperate_time(time): #심야, 새벽, 오전, 오후, 저녁, 야간 ->4시간단위
    if 0<=int(time)<4:
        return '심야'
    elif int(time)<8:
        return '새벽'
    elif int(time)<12:
        return '오전'
    elif int(time)<16:
        return '오후'
    elif int(time)<20:
        return '저녁'
    else:
        return '야간'
concatenated_df['시간구분']=concatenated_df['시간'].apply(seperate_time)
print(concatenated_df['시간구분'].value_counts().sum())
concatenated_df['시간구분'].value_counts()

#### 계절 분류

In [ ]:
def seperate_month(month):
    if int(month) in [12,1,2]:
        return '겨울'
    elif int(month) in [3,4,5]:
        return '봄'
    elif int(month) in [6,7,8]:
        return '여름'
    else:
        return '가을'
concatenated_df['계절']=concatenated_df['월'].apply(seperate_month)
print(concatenated_df['계절'].value_counts().sum())
concatenated_df['계절'].value_counts()

### 병합된 데이터 전처리 확인

In [ ]:
concatenated_df

### 테스트 데이터 전처리 진행

In [ ]:
test=test_data.copy()

In [ ]:
test[['시','구','동']] = test['시군구'].str.split(' ',expand=True)
test.drop(['시군구'],inplace=True, axis=1)

In [ ]:
test[['도로형태1', '도로형태2']] = test['도로형태'].str.split(' - ', expand=True)
test.drop(['도로형태'],inplace=True, axis=1)

In [ ]:
kr_holidays = ['2019-01-01', '2019-02-04', '2019-02-05', '2019-02-06', '2019-03-01', '2019-05-05', '2019-05-12', '2019-06-06', '2019-08-15', '2019-09-12', '2019-09-13', '2019-09-14', '2019-10-03', '2019-10-09', '2019-12-25',
                '2020-01-01' ,'2020-01-24' ,'2020-01-25', '2020-01-26', '2020-03-01', '2020-04-30', '2020-05-05', '2020-06-06', '2020-08-15', '2020-08-17', '2020-09-30', '2020-10-01', '2020-10-02', '2020-10-03', '2020-10-09', '2020-12-25',
                '2021-01-01' ,'2021-02-11' ,'2021-02-12', '2021-02-13', '2021-03-01', '2021-05-05', '2021-05-19', '2021-06-06', '2021-08-15', '2021-09-20', '2021-09-21', '2021-09-22', '2021-10-03', '2021-10-09', '2021-12-25',
                '2022-01-01' ,'2022-01-31' ,'2022-02-01', '2022-02-02', '2022-03-01', '2022-05-05', '2022-05-08', '2022-06-06', '2022-08-15', '2022-09-09', '2022-09-10', '2022-09-11', '2022-09-12', '2022-10-03', '2022-10-09', '2020-10-10', '2022-12-25',
                '2023-01-01' ,'2023-01-21' ,'2023-01-22', '2023-01-23', '2023-01-24', '2023-03-01', '2023-05-05', '2023-05-26', '2023-06-06', '2023-08-15', '2023-09-28', '2023-09-29', '2023-09-30', '2023-10-03', '2023-10-09', '2023-12-25']


In [ ]:
test['사고일시'] = pd.to_datetime(test['사고일시'])

# datetime에서 year, month, day, hour 추출하여 새로운 열 생성
test['년도'] = test['사고일시'].dt.year
test['월'] = test['사고일시'].dt.month
test['day'] = test['사고일시'].dt.day
test['시간'] = test['사고일시'].dt.hour
test['day_of_week'] = test['사고일시'].dt.dayofweek

test['holiday'] = np.where((test.day_of_week >= 5) | (test.사고일시.dt.strftime('%Y-%m-%d').isin(kr_holidays)), '휴일', '평일')

test.drop(columns=['사고일시'], inplace=True)
test.drop(['day', 'day_of_week'], inplace=True, axis=1)

In [ ]:
test['시간구분']=test['시간'].apply(seperate_time)

In [ ]:
test['계절']=test['월'].apply(seperate_month)
test['계절'].value_counts()

In [ ]:
test.drop('ID',axis=1,inplace=True)

#### 테스트 데이터 전처리 결과 확인

In [ ]:
test

#### 합쳐진 훈련 데이터 셋 오류 수정

In [ ]:
concatenated_df = concatenated_df[~concatenated_df['기상상태'].str.split().str[-1].str.contains('안개')]
concatenated_df = concatenated_df[~concatenated_df['노면상태'].str.split().str[-1].str.contains('해빙')]

In [ ]:
concatenated_df

#### 전처리 데이터 저장 - 전처리 매번 로딩할 필요 없다

In [ ]:
# concatenated_df.to_csv(path + f'/데이터들/인코딩 없음/train.csv')
# test.to_csv(path + f'/데이터들/인코딩 없음/test.csv')

In [ ]:
# path = os.getcwd()

# concatenated_df.to_csv(path + f'/데이터들/인코딩 없음/train.csv')
# test = pd.read_csv(path + f'/데이터들/인코딩 없음/test.csv')

#전처리 - 원핫인코딩

In [ ]:
total_data = pd.concat([concatenated_df, test])
total_data = pd.get_dummies(total_data)

In [ ]:
for c in total_data.columns:
    print(c)
print(len(total_data.columns))

In [ ]:
train_data = total_data[~total_data['ECLO'].isnull()]
test_data = total_data[total_data['ECLO'].isnull()]

In [ ]:
test_data = test_data.drop(['사망자수','중상자수','경상자수','부상자수','ECLO'], axis=1)

In [ ]:
train_data

In [ ]:
test_data

In [ ]:
train_data.info()

#AutoML을 이용한 하이퍼파라마터 튜닝

In [ ]:
!pip install optuna

In [ ]:
import optuna
from optuna import Trial
from optuna.samplers import TPESampler
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_log_error as msle


label값 분리

In [ ]:
y1=train_data[['ECLO']]
y2=train_data[['사망자수']]
y3=train_data[['중상자수']]
y4=train_data[['경상자수']]
y5=train_data[['부상자수']]
X=train_data.drop(columns=['사망자수','중상자수','경상자수','부상자수','ECLO'])

훈련,테스트(검증)셋 분리

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y1, test_size=0.2, random_state=42)

XGBRegressor

In [ ]:
def objective(trial: Trial):
    params_xgb={
        'random_state':42,
        'max_depth':trial.suggest_int('max_depth',3,10),                        #트리의 최대 깊이 3~10
        'learning_rate':trial.suggest_float('learning_rate',0.0001,0.1),        #학습률 0.0001~0.1
        'n_estimators':trial.suggest_int('n_estimators',100,1000,100),          #트리 개수 100~1000 100단위
        'reg_alpha':trial.suggest_float('reg_alpha',0.01,1.0),                  #L1규제
        'reg_lambda':trial.suggest_float('reg_lambda',0.01,1.0),                #L2규제
        "subsample": trial.suggest_float("subsample", 0.7, 1.0),                #학습 시 사용할 훈련 데이터의 비율
        'eta': trial.suggest_float('eta', 0.007, 0.013),                        #부스팅 각 단계에서 사용되는 축소 비율
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.4, 1.0),  #트리 구성하는데 사용할 특성의 비율
    }
    model=XGBRegressor(**params_xgb,n_jobs=-1)
    best_xgb=model.fit(X_train,y_train, eval_set=[(X_test,y_test)],eval_metric='rmsle',early_stopping_rounds=100)

    cgb_pred = model.predict(X_test)
    loss=msle(y_test,cgb_pred)

    return np.sqrt(loss)

In [ ]:
sampler = TPESampler(seed=42)
study1 = optuna.create_study(
    study_name="xgb_parameter_opt",
    direction="minimize",
    sampler=sampler,
)

study1.optimize(objective, n_trials=10)
best_xgb_model=XGBRegressor(**study1.best_trial.params,n_jobs=-1)
best_xgb_model.fit(X_train,y_train, eval_set=[(X_test,y_test)], early_stopping_rounds=100, eval_metric='rmsle',verbose=False)
print("Best Score:", study1.best_value)
print("Best trial:", study1.best_trial.params)

In [ ]:
import joblib

# XGBoost 모델 저장
joblib.dump(best_xgb_model, 'best_xgb_model.pkl')


In [ ]:
optuna.visualization.plot_optimization_history(study1)

In [ ]:
#파라미터들관의 관계
optuna.visualization.plot_parallel_coordinate(study1)

lightlgbm

In [ ]:
from lightgbm import LGBMRegressor, early_stopping

In [ ]:
def objective2(trial:Trial):
    params_lgb ={
        'random_state':42,
        'objective': 'regression',
        'metric': 'rmse',
        'num_leaves': trial.suggest_int('num_leaves', 2, 1024, step=1, log=True),   #최대 잎의 수
        'learning_rate': trial.suggest_loguniform("learning_rate", 1e-8, 1e-2),     #학습률
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000,100),           #트리 개수
        'subsample': trial.suggest_loguniform('subsample', 0.4,1),                  #학습 시 사용할 훈련 데이터의 비율
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.4, 1.0),      #트리를 구성하는 데 사용할 특성의 비율
        'reg_alpha':trial.suggest_float('reg_alpha',0.01,1.0),                      #L1규제
        'reg_lambda':trial.suggest_float('reg_lambda',0.01,1.0),                    #L2규제
        'max_depth': trial.suggest_int('max_depth',3, 10),                          #최대 깊이
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),        #리프 노드에서 필요핸 최소 샘플 개수
    }
    model=LGBMRegressor(**params_lgb,n_jobs=-1)
    best_lgbm=model.fit(X_train,y_train, eval_set=[(X_test,y_test)],eval_metric='rmsle', callbacks=[early_stopping(stopping_rounds=100)])

    lgbm_pred = model.predict(X_test)
    loss=msle(y_test,lgbm_pred)

    return np.sqrt(loss)


In [ ]:
sampler = TPESampler(seed=42)
study2 = optuna.create_study(
    study_name="lgbm_parameter_opt",
    direction="minimize",
    sampler=sampler,
)

study2.optimize(objective2, n_trials=10)
best_lgbm_model = LGBMRegressor(**study2.best_trial.params,n_jobs=-1)
best_lgbm_model.fit(X_train,y_train, eval_set=[(X_test,y_test)],eval_metric='rmsle', callbacks=[early_stopping(stopping_rounds=100)])
print("Best Score:", study2.best_value)
print("Best trial:", study2.best_trial.params)

In [ ]:
joblib.dump(best_lgbm_model, 'best_lgbm_model.pkl')

catboost

In [ ]:
!pip install catboost

In [ ]:
from catboost import CatBoostRegressor

In [ ]:
def objective3(trial:Trial):
    params_cat ={
        'iterations':trial.suggest_int("iterations", 100, 1000,100),                            #부스팅 반복 횟수
        'od_wait':trial.suggest_int('od_wait', 500, 2300),                                      #조기 정지를 위한 반복 횟수
        'learning_rate' : trial.suggest_uniform('learning_rate',1e-8, 1),                       #학습률
        'reg_lambda': trial.suggest_uniform('reg_lambda',1e-5,100),                             #L2규제
        'subsample': trial.suggest_uniform('subsample',0.4,1),                                  #각 부스팅 단계에서 사용되는 훈련 데이터의 샘플링 비율
        'random_strength': trial.suggest_uniform('random_strength',10,50),                      #데이터 샘플의 무작위성을 조절
        'depth': trial.suggest_int('depth',1, 15),                                              #트리의 최대 깊이
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf',1,30),                         #리프 노드에 필요한 최소 데이터 수
        'leaf_estimation_iterations': trial.suggest_int('leaf_estimation_iterations',1,15),     #리프 노드의 추정 반복 횟수
        'bagging_temperature' :trial.suggest_loguniform('bagging_temperature', 0.01, 100.00),   #부트스트랩 샘플링 온도
        'colsample_bylevel':trial.suggest_float('colsample_bylevel', 0.4, 1.0),                 #각 트리 수준에서 특성을 샘플링하는 비율
    }
    model=CatBoostRegressor(**params_cat)
    best_cat=model.fit(X_train, y_train, eval_set=[(X_test,y_test)],early_stopping_rounds=100)

    cat_pred = model.predict(X_test)
    loss=msle(y_test,cat_pred)

    return np.sqrt(loss)


In [ ]:
sampler = TPESampler(seed=42)
study3 = optuna.create_study(
    study_name="cat_parameter_opt",
    direction="minimize",
    sampler=sampler,
)

study3.optimize(objective3, n_trials=10)
best_cat_model=CatBoostRegressor(**study3.best_trial.params)
best_cat_model.fit(X_train,y_train, eval_set=[(X_test,y_test)], early_stopping_rounds=100,verbose=False)
print("Best Score:", study3.best_value)
print("Best trial:", study3.best_trial.params)

In [ ]:
joblib.dump(best_cat_model, 'best_cat_model.pkl')

모델 앙상블 가중치 찾기

In [ ]:
def objective(trial):
    # 가중치 탐색 공간 정의
    w1 = trial.suggest_float('w1', 0.0, 1.0)
    w2 = trial.suggest_float('w2', 0.0, 1.0)
    w3 = trial.suggest_float('w3', 0.0, 1.0)

    # 모델 예측 값의 가중 평균
    y_pred = w1 * best_xgb_model.predict(X_test) + w2 * best_lgbm_model.predict(X_test) + w3 * best_cat_model.predict(X_test)

    loss=msle(y_test,y_pred)

    return np.sqrt(loss)

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

# 최적의 하이퍼파라미터 값 출력
best_weights = study.best_params
print("Best weights:", best_weights)

검증set 성능

In [ ]:
from sklearn.metrics import mean_squared_log_error
def rmsle(y, pred):
    return np.sqrt(mean_squared_log_error(y, pred))

In [ ]:
y_pred_valid=best_weights['w1'] * best_xgb_model.predict(X_test) + best_weights['w2'] * best_lgbm_model.predict(X_test) + best_weights['w3'] * best_cat_model.predict(X_test)
rmsle1 = rmsle(y_test, y_pred_valid)
rmsle1

가중치를 이용해 예측값 계산

In [ ]:
y_pred = best_weights['w1'] * best_xgb_model.predict(test_data) + best_weights['w2'] * best_lgbm_model.predict(test_data) + best_weights['w3'] * best_cat_model.predict(test_data)

In [ ]:
y_pred

파일제출

In [ ]:
sample_submission=pd.read_csv('/content/drive/MyDrive/camp/project1/open/sample_submission.csv')
submit=sample_submission.copy()

In [ ]:
submit['ECLO']=y_pred

In [ ]:
submit

In [ ]:
submit.to_csv('submision.csv', index=False)